In [1]:
from tmdbv3api import TMDb, Movie
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances,cosine_similarity


from openpyxl import load_workbook
tmdb = TMDb()
tmdb.api_key = 'b25be5b88ea4b2eb3e8a36b3ed714eeb'

tmdb.language = 'en'
tmdb.debug = True

In [2]:
# movie = Movie()

In [3]:
# att = ['title','budget','original_language','popularity','revenue','runtime','vote_average','vote_count']


In [4]:
# movie_name_list = list(pd.read_excel('HW4_Data_2022.xlsx',sheet_name='DBMI Data').columns)
# movie_name_list.remove('User')

In [5]:
# def get_movie_att(name,att):
#     movie_id = dict(movie.search(name)[0])['id']
#     attr_movie = {i:j for i,j in dict(movie.details(movie_id)).items() if i in  att}
#     attr_movie = pd.DataFrame(attr_movie.values(),index =attr_movie.keys()).transpose()
    
#     return attr_movie

In [6]:
# dict(movie.details(245554))

In [7]:
# a_list = []
# for _ in movie_name_list:
#     a_list.append(get_movie_att(_,att))

# movie_attr_df = pd.concat(a_list).reset_index(drop= True)

In [8]:
# movie_attr_df[att].to_excel("movie_attrs.xlsx",sheet_name="attrs")

In [9]:
# movie_attr_df[att]

In [10]:
response_df = pd.read_excel('HW4_Data_2022.xlsx',sheet_name='Form Responses 1')



In [11]:
a_list_of_cols = ['Serial Number', 'First Name',
       'Last Name',
       'Section Number',
       'Group Number',
       'The Social Network', 'A Prophet', 'Amour',
       "The King's Speech]", 'La La Land]', ' [Boyhood]', ' [Inception]',
       'A Separation]', ' [The Artist]', ' [The White Ribbon]',
       'Zero Dark Thirty]', ' [Avatar]', ' [Spotlight]', ' [Precious]',
       'The Tree of Life]', ' [12 Years a Slave]',
       'Blue is the Warmest Colour]', ' [Son of Saul]', ' [Up in the Air]',
       'Inglourious Basterds]', ' [Mad Max: Fury Road]', ' [Moonlight]',
       'Birdman]', ' [Manchester by the Sea]', ' [Lincoln]', ' [Hugo]',
       'Toni Erdmann]', ' [The Shape of Water]',
       'Three Billboards Outside Ebbing, Missouri]', ' [Argo]', ' [Gravity]',
       'Black Swan]', ' [Ida]', ' [Leviathan]', ' [The Wolf of Wall Street]',
       'True Grit]', ' [The Descendants]', ' [The Secret in Their Eyes]',
       'Life of Pi]', ' [Arrival]', ' [Call Me by Your Name]',
       "Winter's Bone]", ' [The Grand Budapest Hotel]', ' [Dunkirk]',
       'Inside Llewyn Davis]', ' [A Serious Man]', ' [Toy Story 3]',
       'Beasts of the Southern Wild]', ' [The Imitation Game]',
       'The Fighter]']

In [12]:
def clean_name(x):
    x = x.replace(" [","")
    x = x.replace("]","")

    return x

In [13]:
response_df.columns = [ clean_name(x) for x in a_list_of_cols]
response_df['First Name'] = response_df['First Name'] + response_df['Last Name'] 
g3_response_df = response_df.copy()

g3_response_df.replace("Did Not Watch",np.NaN,inplace=True)
g3_response_df.set_index('First Name',inplace = True)
g3_response_new = g3_response_df.iloc[:,list(range(4,54))].transpose()

In [14]:
# g3_response_df = response_df[response_df['Group Number'] == 3].reset_index(drop=True)

In [15]:
# users

In [16]:
def z_adjust (df):
    mean_df = df.mean()
    sd_df = df.std()
    df = (df - mean_df)/sd_df
    
    return df,mean_df,sd_df

In [17]:
def mean_adjust (df):
    return ((df - df.mean()),df.mean())

In [18]:
def get_csimilarity(df,users):
    cosine_similarity_user = round(pd.DataFrame(cosine_similarity(df),
                          index = users,
                          columns = users)
              ,3)
    return cosine_similarity_user

In [19]:
def get_predictions(df,user_name,users,cosine_similarity_user):
    
    temp_user_list = users.copy()
    temp_user_list.remove(user_name)
    
    total_cosine = sum(abs(cosine_similarity_user.loc[user_name,temp_user_list]))
    new_preds = df.loc[:,temp_user_list].fillna(0).dot(cosine_similarity_user.loc[user_name,temp_user_list])
    new_preds = new_preds/total_cosine
    new_preds = pd.DataFrame(new_preds,columns = [user_name])
    
    return new_preds

In [20]:
def get_all_preds(df,cosine_similarity_user,users):
    a_list = []
    for _ in users:
        a_list.append(get_predictions(df,_,users,cosine_similarity_user))

    all_preds = pd.concat(a_list, axis = 1)
    return all_preds

In [21]:
def driver(df,type_name):
    
    if type_name == "mm":
        z_adjusted = z_adjust(df.transpose())
        users = list(df.index).copy()
        cosine_similarity_user = get_csimilarity(z_adjusted[0].transpose().fillna(0),users)
        all_preds = get_all_preds(z_adjusted[0],cosine_similarity_user,users)
        all_preds = (all_preds  * z_adjusted[2].fillna(0)) +  z_adjusted[1].fillna(0)
        
    else:
        z_adjusted = z_adjust(df)
        users = list(df.columns).copy()
        cosine_similarity_user = get_csimilarity(z_adjusted[0].fillna(0).transpose(),users)
        all_preds = get_all_preds(z_adjusted[0],cosine_similarity_user,users)
        all_preds = (all_preds  * z_adjusted[2].fillna(0)) +  z_adjusted[1].fillna(0)
        all_preds = all_preds.transpose()
    all_preds = round(df.transpose().fillna(all_preds),2)
    return all_preds,cosine_similarity_user


In [22]:
def write_data(df,sheet_name):
    path = r"Similarities New.xlsx"

    book = load_workbook(path)
    writer = pd.ExcelWriter(path, engine = 'openpyxl')
    writer.book = book

    df.to_excel(writer, sheet_name = sheet_name)

    writer.close()

In [23]:
out_put_user_user_dfs = driver(g3_response_new,type_name='uu')
out_put_movie_movie_dfs = driver(g3_response_new,type_name='mm')

In [26]:
write_data(out_put_user_user_dfs[0],'uu_z_preds_23')
write_data(out_put_user_user_dfs[1],'uu_z_cosine_23')
write_data(out_put_movie_movie_dfs[0],'mm_z_preds_23')
write_data(out_put_movie_movie_dfs[1],'mm_z_cosine_23')

,The Social Network,A Prophet,Amour,The King's Speech,La La Land,Boyhood,Inception,A Separation,The Artist,The White Ribbon,...,Call Me by Your Name,Winter's Bone,The Grand Budapest Hotel,Dunkirk,Inside Llewyn Davis,A Serious Man,Toy Story 3,Beasts of the Southern Wild,The Imitation Game,The Fighter
First Name,,,,,,,,,,,,,,,,,,,,,
ShachiGovil,4.12,3.62,3.60,3.87,3.85,3.42,4.40,3.14,3.50,3.78,...,4.25,NaN,4.26,4.05,3.29,NaN,3.78,3.50,4.27,3.76
AmyRussell,4.12,3.62,3.60,3.87,3.85,3.42,4.40,3.14,3.50,3.78,...,4.25,NaN,4.26,4.05,3.29,NaN,3.78,3.50,4.27,3.76
CamilleMack,4.12,3.62,3.60,3.87,3.85,3.42,4.40,3.14,3.50,3.78,...,4.25,NaN,4.26,4.05,3.29,NaN,3.78,3.50,4.27,3.76
TeaganTowhey,4.22,3.66,3.68,3.94,3.98,3.41,5.00,3.07,3.56,3.83,...,4.29,NaN,4.31,4.16,3.29,NaN,3.97,3.59,4.32,3.81
AnushkaShah,4.00,3.63,3.62,3.96,3.96,3.45,5.00,3.16,3.54,3.80,...,4.31,NaN,4.31,5.00,3.32,NaN,3.00,3.55,4.32,3.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XinyuLiu,4.00,3.55,3.00,4.00,3.00,3.44,4.04,3.31,3.31,3.00,...,4.00,NaN,4.00,3.00,3.00,NaN,3.00,3.17,4.00,3.00
ArjunRemeshkumar Nair,5.00,3.67,3.69,4.00,4.00,3.00,5.00,3.24,3.64,3.86,...,4.33,NaN,5.00,5.00,3.51,NaN,4.00,3.67,5.00,3.92
HarshkritiKaur,4.18,3.62,3.60,3.84,1.00,3.37,5.00,3.20,3.46,3.73,...,4.18,NaN,4.22,3.00,3.28,NaN,3.76,3.44,4.25,3.73


In [ ]:
# Movie Movie randome predict 

In [27]:
hist_response_df = pd.read_excel("HW4_Data_2022.xlsx",sheet_name="DBMI Data")

In [28]:
def clean_col(x):
    if x[0] == " ":
        return x[1:]
    else:
        return x

In [29]:
colnames = ['User', ' The Social Network', ' A Prophet', ' Amour',
       " The King's Speech", ' La La Land', ' Boyhood', ' Inception',
       ' A Separation', ' The Artist', ' The White Ribbon',
       ' Zero Dark Thirty', ' Avatar', ' Spotlight', ' Precious',
       ' The Tree of Life', ' 12 Years a Slave', ' Blue is the Warmest Colour',
       ' Son of Saul', ' Up in the Air', ' Inglourious Basterds',
       ' Mad Max: Fury Road', ' Moonlight', ' Birdman',
       ' Manchester by the Sea', ' Lincoln', ' Hugo', ' Toni Erdmann',
       ' The Shape of Water', ' Three Billboards Outside Ebbing, Missouri',
       ' Argo', ' Gravity', ' Black Swan', ' Ida', ' Leviathan',
       ' The Wolf of Wall Street', ' True Grit', ' The Descendants',
       ' The Secret in Their Eyes', ' Life of Pi', ' Arrival',
       ' Call Me by Your Name', " Winter's Bone", ' The Grand Budapest Hotel',
       ' Dunkirk', ' Inside Llewyn Davis', ' A Serious Man', ' Toy Story 3',
       ' Beasts of the Southern Wild', ' The Imitation Game', ' The Fighter']

In [30]:
hist_response_df.columns = [clean_col(x) for x in colnames]

In [31]:
hist_df = hist_response_df.copy()

In [32]:
hist_df.set_index('User',inplace = True)

In [33]:
hist_df = hist_df.transpose()

In [34]:
out_put_user_user_hist_dfs = driver(hist_df,type_name='uu')
out_put_movie_movie_hist_dfs = driver(hist_df,type_name='mm')

In [36]:
write_data(out_put_movie_movie_hist_dfs[0],'mm_z_preds_hist')
write_data(out_put_movie_movie_hist_dfs[1],'mm_z_cosine_hist')

In [38]:

# write_data(out_put_user_user_hist_dfs[0],'uu_z_preds_hist')
# write_data(out_put_user_user_hist_dfs[1],'uu_z_cosine_hist')

In [ ]:
# User User proper Prediction

In [55]:
hist_new = hist_df.transpose().copy()

In [56]:
hist_new= hist_new[~(hist_new["Winter's Bone"].isna() & hist_new["A Serious Man"].isna() & hist_new["Son of Saul"].isna())]


In [57]:
hist_new= pd.concat([g3_response_new.transpose(),hist_new]).transpose()

In [58]:
out_put_uu_hist_con_dfs = driver(hist_new,type_name='uu')
out_put_mm_histcon_dfs = driver(hist_new,type_name='mm')

In [59]:

write_data(out_put_uu_hist_con_dfs[0],'uu_z_preds_hc')
write_data(out_put_uu_hist_con_dfs[1],'uu_z_cosine_hc')
write_data(out_put_mm_histcon_dfs[0],'mm_z_preds_hc')
write_data(out_put_mm_histcon_dfs[1],'mm_z_cosine_hc')

In [326]:
test = pd.read_excel("test_data.xlsx")

In [327]:
test.set_index('user', inplace = True)

In [328]:
test

,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5,Movie 6,Movie 7,Movie 8,Movie 9,Movie 10
user,,,,,,,,,,
U1,NaN,5.0,5.0,5.0,NaN,NaN,NaN,5.0,4.0,NaN
U2,5.0,4.0,5.0,5.0,5.0,5.0,NaN,NaN,NaN,4.0
U3,NaN,NaN,NaN,5.0,NaN,4.0,NaN,NaN,4.0,NaN
U4,3.0,NaN,5.0,NaN,2.0,5.0,5.0,5.0,3.0,5.0
U5,NaN,5.0,5.0,5.0,5.0,4.0,4.0,4.0,4.0,5.0
U6,NaN,5.0,NaN,5.0,4.0,NaN,NaN,NaN,NaN,4.0
U7,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
U8,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,4.0,NaN
U9,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0


In [140]:
round(pd.DataFrame(cosine_similarity(test.fillna(0))),3)

,0,1,2,3,4,5,6,7,8,9
0,1.000,0.519,0.504,0.475,0.750,0.513,0.464,0.469,0.000,0.000
1,0.519,1.000,0.476,0.625,0.784,0.714,0.399,0.246,0.511,0.547
2,0.504,0.476,1.000,0.350,0.549,0.366,0.662,0.261,0.000,0.454
3,0.475,0.625,0.350,1.000,0.792,0.255,0.000,0.477,0.451,0.438
4,0.750,0.784,0.549,0.792,1.000,0.723,0.364,0.546,0.227,0.437
5,0.513,0.714,0.366,0.255,0.723,1.000,0.552,0.272,0.276,0.227
6,0.464,0.399,0.662,0.000,0.364,0.552,1.000,0.000,0.000,0.000
7,0.469,0.246,0.261,0.477,0.546,0.272,0.000,1.000,0.000,0.317
8,0.000,0.511,0.000,0.451,0.227,0.276,0.000,0.000,1.000,0.000
9,0.000,0.547,0.454,0.438,0.437,0.227,0.000,0.317,0.000,1.000
